In [1]:
import os
import os.path as osp
import argparse
from cv2 import imread, normalize, resize
import cv2
import json
import numpy as np
import time
import datetime
import random
import logging
import sys
from PIL import Image
from pytorchtools import EarlyStopping # 위 링크의 깃허브 파일에서 임포트
from utils import euclidean_metric, one_hot, count_acc
from utils import pprint, set_gpu, ensure_path, AverageMeter, Timer, accuracy, one_hot

import torch
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
from torch import nn, optim, autograd
from torch.utils.data import DataLoader
import torch.utils.data as data
import torchvision
from random import sample, random

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, ConcatDataset, Dataset, TensorDataset, WeightedRandomSampler
import os.path as osp
from tqdm import tqdm

from random import sample, random

import clip
import skimage
import IPython.display
import matplotlib.pyplot as plt

from collections import OrderedDict
import torch

batch_size = 32
num_epoch = 100

/home/iai/anaconda3/envs/sh_clip/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

zero_shot_model, zero_shot_preprocess= clip.load("RN50", device=device)

# Freeze the model
for param in zero_shot_model.parameters():
    param.requires_grad = False
    
fine_tunning_model ,fine_tunning_preprocess= clip.load("RN50", device=device)

In [3]:
class Customdataset(data.Dataset):
    def __init__(self,transform = None):
        dalist=None
        self.file_paths = []
        self.label = []
        self.concept = []
        self.transform = transform

        folder_list = ["custom_blur","custom_brightness","custom_cameraz","custom_lightness"]
        
        for folder_name in folder_list:
            path_dir = "/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/augmented_image2/train1-1/" +  folder_name
            file_list = os.listdir(path_dir)
            
            for file_name in file_list:
                self.file_paths.append(path_dir + '/' + file_name)
                self.label.append(file_name.split('_')[-1][0])
                
                if str(file_name.split('_')[2]) == 'cameraz':
                    self.concept.append(3)
                    
                elif str(file_name.split('_')[2]) == 'blur':
                    self.concept.append(3)
                    
                elif str(file_name.split('_')[2]) == 'brigntness':
                    self.concept.append(2)
                    
                    
                elif str(file_name.split('_')[2]) == 'lightness':
                    self.concept.append(1)

        
        
        file_path = '/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/imageset/single_image.2class.subject-split.trainval-repeat0/fold.5-5/ratio/100%/train.1-1.txt' # 폴더 경로

        with open(file_path) as f:
            lines = f.readlines()

        image_names = [line.rstrip('\n') for line in lines]
        
                
        with open('/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/annotation/single_image.6class.json', 'r') as json_file:
            data = json.load(json_file)
            
        path_image = "/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/image/"
        files = os.listdir(path_image)
        
        for file in files:
            if file in image_names:
                self.label.append(data['single_image'][file]['class'][0])
                self.file_paths.append(path_image + '/' + file)
                self.concept.append(0)
        
        
                
        file_path = '/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/imageset/single_image.2class.subject-split.trainval-repeat0/fold.5-5/ratio/100%/validation.1-1.txt' # 폴더 경로

        for folder_name in folder_list:
            path_dir = "/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/augmented_image2/train1-1/" +  folder_name
            file_list = os.listdir(path_dir)
            
            for file_name in file_list:
                self.file_paths.append(path_dir + '/' + file_name)
                self.label.append(file_name.split('_')[-1][0])
                
                if str(file_name.split('_')[2]) == 'cameraz':
                    self.concept.append(3)
                    
                elif str(file_name.split('_')[2]) == 'blur':
                    self.concept.append(3)
                    
                elif str(file_name.split('_')[2]) == 'brigntness':
                    self.concept.append(2)
                    
                    
                elif str(file_name.split('_')[2]) == 'lightness':
                    self.concept.append(1)
        
        with open(file_path) as f:
            lines = f.readlines()

        image_names = [line.rstrip('\n') for line in lines]
        
                
        with open('/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/annotation/single_image.6class.json', 'r') as json_file:
            data = json.load(json_file)
            
        path_image = "/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/image/"
        files = os.listdir(path_image)
        
        for file in files:
            if file in image_names:
                self.label.append(data['single_image'][file]['class'][0])
                self.file_paths.append(path_image + '/' + file)
                self.concept.append(0) 

    def __len__(self):

        return len(self.file_paths)

    def __getitem__(self, idx):
        
        label = self.label[idx]
        idx_num = idx
        tmp_prob = np.random.rand()
        weight_dict = {"0" :  (400/250) / (400/250 + 400 * 5 / 30), "1" : (400/30) / (400/250 + 400 * 5 / 30), "2" : (400/30) / (400/250 + 400 * 5 / 30), 
                       "3" : (400/30) / (400/250 + 400 * 5 / 30), "4" : (400/30) / (400/250 + 400 * 5 / 30), "5" : (400/30) / (400/250 + 400 * 5 / 30)}        
        
        if weight_dict[str(label)] > tmp_prob:
            while True : 
                idx_num = np.random.randint(len(self.file_paths))
                tmp_prob =  np.random.rand()
                label = self.label[idx_num]
                
                if weight_dict[str(label)] < tmp_prob:
                    break
        
        
        path = self.file_paths[idx_num]
        img = Image.open(path).convert('RGB')
        image = img.crop((30,48,482,235))
        concept = self.concept[idx_num]
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, int(label), int(concept)

In [4]:
data_transforms = transforms.Compose([
        
        transforms.Resize(size=224),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),

        ])

##커스텀 데이터셋
dataset = Customdataset(transform=data_transforms)
#dataset = Customdataset(transform=fine_tunning_preprocess)
train_size = int(0.8 * len(dataset))
print(f"train size :{train_size}")
val_size = len(dataset) - train_size
print(f"test_size :{val_size}")
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

print(f"train_dataset :{len(train_dataset)}")
print(f'test dataset :{len(val_dataset)}')


train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size = batch_size, #배치사이즈
                                               shuffle = True 
                                               )

val_loader = torch.utils.data.DataLoader(val_dataset,
                                               batch_size = batch_size,                                              
                                               shuffle = True 
                                              )

train size :8576
test_size :2144
train_dataset :8576
test dataset :2144


In [5]:

"""
model, preprocess = clip.load("RN50", device=device)
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)
"""

'\nmodel, preprocess = clip.load("RN50", device=device)\nmodel.cuda().eval()\ninput_resolution = model.visual.input_resolution\ncontext_length = model.context_length\nvocab_size = model.vocab_size\n\nprint("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")\nprint("Input resolution:", input_resolution)\nprint("Context length:", context_length)\nprint("Vocab size:", vocab_size)\n'

In [6]:
zero_shot_preprocess

Compose(
    Resize(size=224, interpolation=PIL.Image.BICUBIC)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x7f1f7a8e2c10>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [7]:
import torch.distributions as distributions

def custom_loss(zero_shot_output_context, fine_output_context, fine_output_label, target, alpha):

    kl_loss = F.kl_div(zero_shot_output_context, fine_output_context, reduction = 'batchmean').requires_grad_(True)
    ce_loss = F.cross_entropy(fine_output_label, target)
    
    final_loss = (ce_loss + alpha * kl_loss)
    #print(final_loss)
    
    return final_loss

def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float()

In [8]:
x = torch.tensor([[1.0, 2.0],[3.0, 4.0]], requires_grad = True)
x2 =  torch.tensor([[1.0, 2.0],[3.0, 4.0]], requires_grad = True)
# tensor([[1., 2.],
#         [3., 4.]], requires_grad=True)

y = x + 2
# tensor([[3., 4.],
#         [5., 6.]], grad_fn=<AddBackward0>)

z = y * y * 3
# tensor([[ 27.,  48.],
#         [ 75., 108.]], grad_fn=<MulBackward0>)

out = z.mean()
# tensor(64.5000, grad_fn=<MeanBackward0>)
#out = F.kl_div(x, x2)

out.backward()
print(x.grad)
# tensor([[4.5000, 6.0000],
#         [7.5000, 9.0000]])

tensor([[4.5000, 6.0000],
        [7.5000, 9.0000]])


In [9]:
model_path = '/home/iai/Desktop/SH/cognex/model/'
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, fine_tunning_model.parameters()), lr=0.05, momentum=0.9, nesterov=True, weight_decay= 5 * 1e-2) 
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 100)

#optimizer = torch.optim.Adam(fine_tunning_model.parameters(), lr=0.00005)
optimizer2 = torch.optim.Adam(zero_shot_model.parameters(), lr=0.00005)
#torch.autograd.set_detect_anomaly(True)
#optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.05, momentum=0.9, nesterov=True, weight_decay=0.0005) 
#lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 100)

cross_entropy_loss_func = nn.CrossEntropyLoss()
early_stopping = EarlyStopping(patience = 4, verbose = True, min_epoch=50)

global_count = 0
device = torch.device('cuda:0')

train_loss_arr =[]
val_loss_arr =[]
n = len(train_loader)

label_list =['ok', 'scratch', 'fm', 'pin', 'dent', 'glue']
context_list = ['repeat', 'lightness' , 'brigntness' , 'blur', 'cameraz']

label_text = []
for label in label_list:
    label_text.append("a photo of " + str(label))
    
context_label_text = []
for label in label_list:
    for context in context_list:
        context_label_text.append("a " +str(context) + " of " + str(label))

label_text = clip.tokenize(label_text).to(device)
context_label_text = clip.tokenize(context_label_text).to(device)


torch.autograd.set_detect_anomaly(False)
for epoch in range(1, num_epoch + 1):
    
    fine_tunning_model.train()
    train_running_loss = 0.0
    train_running_accuracy = 0.0
    total = 0 
       
    for i,[image,label, context] in enumerate(train_loader):
        
        target = label.to(device)
        input_image = image.to(device)
        
        
        with torch.no_grad():
            zero_shot_image_features = zero_shot_model.encode_image(input_image).float()
            zero_shot_context_text_features = zero_shot_model.encode_text(context_label_text).float()

                
            #zero_shot_image_features /= zero_shot_image_features.norm(dim=-1, keepdim=True)
            #zero_shot_context_text_features /= zero_shot_context_text_features.norm(dim=-1, keepdim=True)

            zero_shot_output_context = (zero_shot_image_features @ zero_shot_context_text_features.T).softmax(dim=-1)

        
        fine_tunning_image_features = fine_tunning_model.encode_image(input_image).float()
        fine_tunning_context_text_features = fine_tunning_model.encode_text(context_label_text).float()
        
        #fine_tunning_image_features /= fine_tunning_image_features.norm(dim=-1, keepdim=True)
        #fine_tunning_context_text_features /= fine_tunning_context_text_features.norm(dim=-1, keepdim=True).requires_grad_(True)
        fine_tunning_output_context = (fine_tunning_image_features @ fine_tunning_context_text_features.T).softmax(dim=-1)
        
        
        fine_tunning_label_text_features = fine_tunning_model.encode_text(label_text).float()
        #fine_tunning_label_text_features /= fine_tunning_label_text_features.norm(dim=-1, keepdim=True).requires_grad_(True).float()
        fine_tunning_output_label = (fine_tunning_image_features @ fine_tunning_label_text_features.T).softmax(dim=-1)
        
        
        loss = custom_loss(zero_shot_output_context, fine_tunning_output_context, fine_tunning_output_label, target, alpha=0.5)
        
        optimizer2.zero_grad()
        optimizer.zero_grad()
        
        loss.backward(retain_graph=True)
        optimizer.step()
        
        '''
        if device == "cpu":
            optimizer.step()
        else : 
            #convert_models_to_fp32(fine_tunning_model)
            optimizer.step()
            clip.fine_tunning_model.convert_weights(fine_tunning_model)
        '''    
        torch.cuda.synchronize()
        train_running_loss += loss.item()
        
        total += target.size(0)
        _, predicted = torch.max(fine_tunning_output_label, 1) 
        train_running_accuracy += (predicted ==target).sum().item() 
        
        
    train_loss_arr.append(train_running_loss / n)
    train_accuracy = (100 * train_running_accuracy / total)
    #lr_scheduler.step()      
    
    fine_tunning_model.eval()
    val_running_loss = 0.0
    val_running_accuracy = 0.0
    total = 0 
    
    with torch.no_grad():
        for j,[image,label, context] in enumerate(val_loader):
            
            target = label.to(device)
            input_image = image.to(device)
            
            fine_tunning_image_features = fine_tunning_model.encode_image(input_image).float()
            fine_tunning_label_text_features = fine_tunning_model.encode_text(label_text).float()
            fine_tunning_output_label = (fine_tunning_image_features @ fine_tunning_label_text_features.T).softmax(dim=-1)
            
            loss = cross_entropy_loss_func(fine_tunning_output_label,target)
            
            total += target.size(0)
            val_running_loss += loss.item()
            _, predicted = torch.max(fine_tunning_output_label, 1) 
            val_running_accuracy += (predicted == target).sum().item() 
        
        val_loss_arr.append(val_running_loss / len(val_loader))       
        early_stopping(val_running_loss, fine_tunning_model)
        torch.save(fine_tunning_model, model_path + str(epoch-1) + 'epoch_model.pt')
        
        val_accuracy = (100 * val_running_accuracy / total)  

        print('Epoch: %d , train_loss: %.5f  , train_accuracy: %.2f , val_loss: %.5f , val_accuracy: {%.2f}' 
            %(epoch , train_running_loss / len(train_loader) ,train_accuracy , val_running_loss / len(val_loader) , val_accuracy))

        
        if early_stopping.early_stop: # 조건 만족 시 조기 종료
            break


Validation loss decreased (inf --> 120.047886).  Saving model ...
Epoch: 1 , train_loss: 0.04127  , train_accuracy: 51.70 , val_loss: 1.79176 , val_accuracy: {64.65}
Validation loss decreased (120.047886 --> 120.047886).  Saving model ...
Epoch: 2 , train_loss: 0.07449  , train_accuracy: 66.32 , val_loss: 1.79176 , val_accuracy: {64.88}
Validation loss decreased (120.047886 --> 120.047886).  Saving model ...
Epoch: 3 , train_loss: 0.07449  , train_accuracy: 66.30 , val_loss: 1.79176 , val_accuracy: {65.25}
Validation loss decreased (120.047886 --> 120.047886).  Saving model ...
Epoch: 4 , train_loss: 0.07449  , train_accuracy: 66.03 , val_loss: 1.79176 , val_accuracy: {64.51}
Validation loss decreased (120.047886 --> 120.047886).  Saving model ...
Epoch: 5 , train_loss: 0.07449  , train_accuracy: 66.41 , val_loss: 1.79176 , val_accuracy: {64.83}
Validation loss decreased (120.047886 --> 120.047886).  Saving model ...
Epoch: 6 , train_loss: 0.07449  , train_accuracy: 66.71 , val_loss: 1

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt


plt.plot(train_loss_arr,
         color='skyblue',
         marker='o', markerfacecolor='blue',
         markersize=4)

plt.plot(val_loss_arr,
         color='pink',
         marker='x', markerfacecolor='red',
         markersize=4)

plt.title('Model loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
import torch.utils.data as data

class Customdataset(data.Dataset):
    def __init__(self,transform = None):
        dalist=None
        self.file_paths = []
        self.label = []
        self.concept = []
        self.transform = transform

        file_path = '/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/imageset/single_image.2class.subject-split.trainval-repeat0/fold.5-5/ratio/100%/test.1.txt' # 폴더 경로

        with open(file_path) as f:
            lines = f.readlines()

        image_names = [line.rstrip('\n') for line in lines]


        path = '/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/image' # 폴더 경로
        files = os.listdir(path) # 해당 폴더에 있는 파일 이름을 리스트 형태로 받음
        
        with open('/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/annotation/single_image.6class.json', 'r') as json_file:
            data = json.load(json_file)  
    
        for file in files:
            if file in image_names:
                self.label.append(data['single_image'][file]['class'][0])
                self.file_paths.append(path + '/' + file)
                
                if str(file.split('_')[1]) == 'cameraz':
                    self.concept.append(3)
                    
                elif str(file.split('_')[1]) == 'repeat':
                    self.concept.append(0)
                    
                elif str(file.split('_')[1]) == 'brightness':
                    self.concept.append(2)
                    
                elif str(file.split('_')[1]) == 'lcondition':
                    self.concept.append(1)
                    
        print(len(label))
        print(len(file_path))
        
    def __len__(self):

        return len(self.concept)


    def __getitem__(self, idx):
        path = self.file_paths[idx]
        image = Image.open(path).convert('RGB')
        label = self.label[idx]
        concept = self.concept[idx]
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, int(label)
    
    
test_dataset = Customdataset(transform=data_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                               batch_size = 32                                       
                                              )

In [ ]:
fine_tunning_model ,fine_tunning_preprocess= clip.load("RN50", device=device)

best_model_epoch = val_loss_arr.index(min(val_loss_arr))
fine_tunning_model = torch.load(model_path + str(best_model_epoch) +'epoch_model.pt')
fine_tunning_model.to(device)

In [ ]:
correct = 0
total = 0
pred_ls = []
real_ls = []

context_ls = [] 

with torch.no_grad(): # 파라미터 업데이트 같은거 안하기 때문에 no_grad를 사용.
  # net.eval() # batch normalization이나 dropout을 사용하지 않았기 때문에 사용하지 않음. 항상 주의해야함.
  for data in test_loader:
    target = label.to(device)
    input_image = image.to(device)
    
    fine_tunning_image_features = fine_tunning_model.encode_image(input_image).float()
    fine_tunning_label_text_features = fine_tunning_model.encode_text(label_text).float()
    fine_tunning_output_label = (fine_tunning_image_features @ fine_tunning_label_text_features.T).softmax(dim=-1)
    
    _, predicted = torch.max(fine_tunning_output_label, 1) 
    total += target.size(0) # test 개수
    correct += (predicted == target).sum().item() # 예측값과 실제값이 맞으면 1 아니면 0으로 합산.
  
    pred_ls.append(predicted.cpu().numpy())
    real_ls.append(target.cpu().numpy()) 
  

print(f'accuracy of 1600 test images: {100*correct/total}%')

In [ ]:
pred = np.array(pred_ls).reshape(-1)
real = np.array(real_ls).reshape(-1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(pred, real))

In [ ]:
import itertools

def plot_confusion_matrix(con_mat, labels, title='Confusion Matrix', cmap=plt.cm.get_cmap('Blues'), normalize=False):
    plt.imshow(con_mat, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    marks = np.arange(len(labels))
    nlabels = []
    for k in range(len(con_mat)):
        n = sum(con_mat[k])
        nlabel = '{0}(n={1})'.format(labels[k],n)
        nlabels.append(nlabel)
    plt.xticks(marks, labels)
    plt.yticks(marks, nlabels)

    thresh = con_mat.max() / 2.
    if normalize:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, '{0}%'.format(con_mat[i, j] * 100 / n), horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    else:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, con_mat[i, j], horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
cm_df = confusion_matrix(pred, real)    
label=['ok', 'scratch', 'fm', 'pin', 'dent', 'glue']
plot_confusion_matrix(cm_df, labels=label, normalize=False)